In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re
import pickle

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version

import copy
import itertools
import adjustText

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
from AbsPerfectPower import AbsPerfectPower as appwr 
#-----
from DOVSAudit import DOVSAudit
from DABatch import DABatch
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Utilities_dt
from Utilities_df import DFConstructType
import Plot_General
import Plot_Box_sns
import Plot_Hist
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSingleSlicer as DFSingleSlicer

In [ ]:
lom1 = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\abs_perfect_power\OPCOs_OLD\ap\20231001_20240930\METHOD_2\partials\20231001_20231031.pkl')
lom2 = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\abs_perfect_power\OPCOs_NEWFF\ap\20231001_20240930\METHOD_2\partials\20231001_20231031.pkl')
lom3 = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\abs_perfect_power\OPCOs_NEWFT\ap\20231001_20240930\METHOD_2\partials\20231001_20231031.pkl')
lom4 = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\abs_perfect_power\OPCOs_NEWTT\ap\20231001_20240930\METHOD_2\partials\20231001_20231031.pkl')

In [ ]:
print(lom1.shape)
print(lom2.shape)
print(lom3.shape)
print(lom4.shape)

In [ ]:
dev_df = lom3.copy()

In [ ]:
# dev_df.dropna(subset=['aep_premise_nb'])

In [ ]:
# dev_df = dev_df.iloc[:10000].copy()
# dev_df = pd.concat([dev_df.iloc[:10000], dev_df.iloc[-10000:]])
# dev_df = dev_df.sort_values(by=['serialnumber', 'island_start_starttimeperiod'], ignore_index=True).iloc[:100000]

In [ ]:
dev_df

In [ ]:
start = time.time()
yeet1 = appwr.combine_zero_times_by_PN(
    zero_times_df = dev_df, 
    t_min_col     = 'island_start_starttimeperiod', 
    t_max_col     = 'island_end_starttimeperiod', 
    PN_col        = 'aep_premise_nb',    
)
print(time.time()-start)

In [ ]:
yeet1

In [ ]:
appwr.build_acc_srvc_intrrptns_table_df(
    zero_times_df = dev_df, 
    SN_col        = 'serialnumber'
)

In [ ]:
appwr.build_acc_srvc_intrrptns_table_df(
    zero_times_df = yeet1, 
    SN_col        = 'aep_premise_nb'
)

In [ ]:
lom3.head()

In [ ]:
print(lom1['serialnumber'].nunique())
print(lom2['serialnumber'].nunique())
print(lom3['serialnumber'].nunique())
print(lom4['serialnumber'].nunique())

In [ ]:
lom3.sort_values(by=['serialnumber', 'island_start_starttimeperiod'], ignore_index=True).equals(
lom4.sort_values(by=['serialnumber', 'island_start_starttimeperiod'], ignore_index=True)
)

In [ ]:
vcs_1 = lom1['serialnumber'].value_counts()
vcs_1

In [ ]:
vcs_3 = lom3['serialnumber'].value_counts()
vcs_3

In [ ]:
vcs_1.equals(vcs_3)

In [ ]:
vcs_1_df = vcs_1.to_frame().sort_index().reset_index()
vcs_3_df = vcs_3.to_frame().sort_index().reset_index()

In [ ]:
dffs = Utilities_df.get_dfs_diff(
    vcs_1_df, 
    vcs_3_df
)

In [ ]:
dffs.head(20)

In [ ]:
vcs_1_df.iloc[725]

In [ ]:
lom1[lom1['serialnumber']=='578988259']

In [ ]:
lom3[lom3['serialnumber']=='578988259']

In [ ]:
conn_aws  = Utilities.get_athena_prod_aws_connection()

In [ ]:
save_dir = r'C:\Users\s346557\Documents\LocalData\abs_perfect_power\OPCOs_v1T'

In [ ]:
# opcos = ['ap', 'ky', 'oh', 'im', 'pso', 'swp', 'tx']
opcos = ['ap', 'oh', 'im', 'pso', 'swp']
# opcos = ['oh']

In [ ]:
opco_zt_dfs = {}
for opco_i in opcos:
    save_dir_i = os.path.join(save_dir, opco_i)
    df_i = appwr.concat_zero_times_dfs_in_dir(
        dir_path             = save_dir_i, 
        regex_pattern        = r'\d{8}_\d{8}.*', 
        ignore_case          = False, 
        ext                  = '.pkl', 
        make_col_types_equal = False, 
        cols_to_drop         = [
            'island_number', 
            'island_start_location', 
            'island_end_location'
        ], 
        return_paths         = False
    )
    opco_zt_dfs[opco_i] = df_i


In [ ]:
app_SNs_ap  = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\abs_perfect_power\20231001_20240930\app_SNs_ap.pkl')
app_SNs_im  = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\abs_perfect_power\20231001_20240930\app_SNs_im.pkl')
app_SNs_oh  = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\abs_perfect_power\20231001_20240930\app_SNs_oh.pkl')
app_SNs_pso = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\abs_perfect_power\20231001_20240930\app_SNs_pso.pkl')
app_SNs_swp = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\abs_perfect_power\20231001_20240930\app_SNs_swp.pkl')

In [ ]:
opco_n_perfect_SNs = dict()
opco_n_perfect_SNs['ap']  = len(list(set(app_SNs_ap['serialnumber'].unique()).difference(set(opco_zt_dfs['ap']['serialnumber'].unique()))))
opco_n_perfect_SNs['im']  = len(list(set(app_SNs_im['serialnumber'].unique()).difference(set(opco_zt_dfs['im']['serialnumber'].unique()))))
opco_n_perfect_SNs['oh']  = len(list(set(app_SNs_oh['serialnumber'].unique()).difference(set(opco_zt_dfs['oh']['serialnumber'].unique()))))
opco_n_perfect_SNs['pso'] = len(list(set(app_SNs_pso['serialnumber'].unique()).difference(set(opco_zt_dfs['pso']['serialnumber'].unique()))))
opco_n_perfect_SNs['swp'] = len(list(set(app_SNs_swp['serialnumber'].unique()).difference(set(opco_zt_dfs['swp']['serialnumber'].unique()))))

In [ ]:
acc_srvc_intrrptns_dict = {}
for opco_i in opcos:
    n_perfect_SNs_i = opco_n_perfect_SNs[opco_i]
    acc_srvc_intrrptns_dict[opco_i] = appwr.build_acc_srvc_intrrptns_table_df(
        zero_times_df  = opco_zt_dfs[opco_i], 
        supplmntl_data = {0: opco_n_perfect_SNs[opco_i]}
    )

In [ ]:
acc_outg_drtn_dict = {}
for opco_i in opcos:
    n_perfect_SNs_i = opco_n_perfect_SNs[opco_i]
    #-----
    df_i = appwr.build_acc_outg_drtn_table_df(
        zero_times_df  = opco_zt_dfs[opco_i], 
        bins           = None, 
        supplmntl_data = {(None, 6): n_perfect_SNs_i}, 
        #-----
        SN_col         = 'serialnumber', 
        island_beg_col = 'island_start_starttimeperiod', 
        island_end_col = 'island_end_starttimeperiod', 
        data_freq      = pd.Timedelta('15 min')
    )
    acc_outg_drtn_dict[opco_i] = df_i

In [ ]:
acc_srvc_intrrptns_dict['oh']

In [ ]:
opco_n_perfect_SNs['oh']

In [ ]:
opco_zt_dfs['oh']

In [ ]:
# import AbsPerfectPower
# reload(AbsPerfectPower)
# from AbsPerfectPower import AbsPerfectPower as appwr

In [ ]:
figax =  appwr.histplot_acc_srvc_intrrptns(
    zero_times_df  = opco_zt_dfs['oh'], 
    bins           = None, 
    supplmntl_data = {0: opco_n_perfect_SNs['oh']}, 
    stat           = 'probability', 
    figax          = None, 
    fig_num        = 0, 
    set_logy       = False, 
    title          = None, 
    include_text   = True, 
    #-----
    SN_col         = 'serialnumber', 
)

In [ ]:
# startcol_0 = 0
# startcol_1 = 5
# startrow_i = 0

# with pd.ExcelWriter(r'C:\Users\s346557\Downloads\Test.xlsx') as writer:  
#     for opco_i in opcos:
#         srvc_intrrptns_i = acc_srvc_intrrptns_dict[opco_i].copy()
#         outg_drtns_i     = acc_outg_drtn_dict[opco_i].copy()
#         #-----
#         srvc_intrrptns_i = Utilities_df.prepend_level_to_MultiIndex(df=srvc_intrrptns_i, level_val=f"Acc. Service Interruptions: OPCO = {opco_i}", axis=1)
#         outg_drtns_i     = Utilities_df.prepend_level_to_MultiIndex(df=outg_drtns_i, level_val=f"Acc. Outage Duration: OPCO = {opco_i}", axis=1)
#         #-----
#         srvc_intrrptns_i.to_excel(writer, startrow=startrow_i, startcol=startcol_0)
#         outg_drtns_i.to_excel(     writer, startrow=startrow_i, startcol=startcol_1)
#         startrow_i += np.max([srvc_intrrptns_i.shape[0], outg_drtns_i.shape[0]])+5

In [ ]:
amanda_df = pd.read_csv(r'C:\Users\s346557\Documents\LocalData\abs_perfect_power\fromAmanda\Perfect_Power Week ending Nov 17, 2024.csv')

In [ ]:
amanda_df[amanda_df['State']=='OH']

In [ ]:
yeet1[yeet1['aep_premise_nb']=='73635839']

In [ ]:
set(yeet1['aep_premise_nb'].unique()).intersection(set(amanda_df['Premise'].unique()))